In [ ]:
from __future__ import print_function
import numpy as np
from sklearn.tests.test_cross_validation import test_train_test_split
import pandas as pd
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Convolution1D, MaxPooling1D
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import random

In [ ]:
# Embedding
max_features = 20000
maxlen = 100
embedding_size = 100

# Convolution
filter_length = 2
nb_filter = 10
pool_length = 2

# LSTM
lstm_output_size = 5

# Training
batch_size = 30
nb_epoch = 2
test__train_split_ratio = 0.1# 0.1 means 10% test

In [ ]:
def split_train_test(corpus, labels):
	# Randomize the dataSet
	random_data = []
	random_labels = []
	
	# Sample indices from 0..len(corpus)
	size = len(corpus)
	rand_indices = random.sample(range(size), size)
	
	
	# Insert in the final dataset N=self.datasetSize random tweets from the rawData
	for index in rand_indices:
		random_data.append(corpus[index])
		random_labels.append(labels[index])
	
	# Calculate the test set size
	test_set_size = int(test__train_split_ratio * size)
	
	# The trainSet starts from the begining until before the end by the test set size 
	train_set = random_data[0 : size - test_set_size]
	test_set  = random_data[len(train_set) : size]
	train_set_labels = random_labels[0 : size - test_set_size]
	test_set_labels  = random_labels[len(train_set) : size]	
	return train_set, train_set_labels, test_set, test_set_labels


In [ ]:
def corpus_to_indices(text):
	#words_map = build_words_map(text)
		
	#text_to_indices(text, words_map)

	# The vocabulary map
	words_map = {}

	# Index of words
	index = 0
	
	# Initialize the output list
	text_indices = []
	maxlen = 0
	# Loop line by line
	
	for line in text:

		# Split into words	
		try:		
			line = str(line)
			line_words = line.split()
		except:
			print(str(line))
	
		if len(line_words) > maxlen:
			maxlen = len(line_words) 
		# Initialize the line_indices
		line_indices = []
		# Loop word by word
		for word in line_words:
			# Store the word once in the wordMap
			if not word in words_map:
				words_map[word] = index
				# Increment the index for the next word
				index += 1

			# Add the index to the line_indices
			line_indices.append(words_map[word])

		# Add the line_indices to the output list
		text_indices.append(line_indices)


	return text_indices, len(words_map), maxlen



In [ ]:
def load_data():

	# Load training data frame
	train_data_path = '..\\..\\dat\\train.tsv'
	train_df = pd.read_table(train_data_path)
	print('Training DataFrame loaded')
	
	text = train_df['item_description']
	text_indices, voc_size, maxlen = corpus_to_indices(text)
	
	# Load labels
	labels = np.array(train_df['price'])
	
	#X_train, y_train, X_test, y_test = split_train_test(text_indices, labels) 
    #return X_train, y_train, X_test, y_test, voc_size, maxlen
	return text_indices, labels, voc_size, maxlen


In [ ]:
print('Loading data...')

X, Y, max_features, maxlen = load_data()




In [ ]:
print('Pad sequences (samples x time)')
X = sequence.pad_sequences(X, maxlen=maxlen)



In [ ]:
print('X shape:', X.shape)


Y = np.reshape(Y, [len(Y), 1])


print('Y shape:', Y.shape)



In [ ]:


print('Build model...')
def model():
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))

    model.add(Convolution1D(nb_filter=nb_filter,
                            filter_length=filter_length,
                            border_mode='valid',
                            activation='relu',
                            subsample_length=1))
    model.add(MaxPooling1D(pool_length=pool_length))

    model.add(LSTM(lstm_output_size))

    model.add(Dense(1))
    #model.add(Activation('sigmoid'))

    import keras.optimizers
    opt = keras.optimizers.adam(0.01)

    model.compile(loss='mean_squared_error',
                  optimizer=opt,
                  metrics=['accuracy'])
    model.save('CNN_LSTM_no_glove_KFold_model', overwrite=True)
    
    return model


In [ ]:
# fix random seed for reproducibility
print('Train...')
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=model, nb_epoch=100, batch_size=5, verbose=0)


In [ ]:

kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))